In [1]:
import cv2
import numpy as np
import os

from keras.models import Sequential
from keras.layers import MaxPooling2D, Dropout, Flatten, Dense, Conv2D
from sklearn import metrics
import tensorflow as tf

### Load Training Data

In [2]:
train_dir = r'C:\Users\elsayed\Projects\AI\Skin cancer\data\train'

In [3]:
categories = ['benign', 'malignant']
training_data = []
img_size = 50

In [4]:
for category in categories:
    path = os.path.join(train_dir, category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img))
            new_array = cv2.resize(img_array, (125, 100))
            training_data.append([new_array, class_num])
        except Exception as ex:
            pass

In [5]:
len(training_data)

2637

In [6]:
x = []
y = []

for feature, label in training_data:
    x.append(feature)
    y.append(label)

In [7]:
x = np.array(x)
y = np.array(y)

### Load Testing Data

In [8]:
test_dir = r'C:\Users\elsayed\Projects\AI\Skin cancer\data\test'

In [9]:
test_categories = ['benign', 'malignant']
testing_data = []

In [10]:
for category_test in test_categories:
    test_path = os.path.join(test_dir, category_test)
    class_num_test = test_categories.index(category_test)
    for test_img in os.listdir(test_path):
        try:
            test_img_array = cv2.imread(os.path.join(test_path, test_img))
            test_new_array = cv2.resize(test_img_array, (125, 100))
            testing_data.append([test_new_array, class_num_test])
        except Exception as ex:
            pass

In [11]:
len(testing_data)

660

In [12]:
x_test = []
y_test = []

for feature, label in testing_data:
    x_test.append(feature)
    y_test.append(label)

In [13]:
x_test = np.array(x_test)
y_test = np.array(y_test)

### The Model

In [14]:
with tf.device ('/gpu:1'):

    input_shape = (100, 125, 3)
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
    model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.16))

    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same'))
    model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.20))

    model.add(Conv2D(64, (3, 3), activation='relu',padding = 'same'))
    model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 125, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 125, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 50, 62, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 50, 62, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 62, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 50, 62, 32)        9248      
                                                        

In [15]:
model.compile(optimizer = "adam" , loss = "binary_crossentropy", metrics=["accuracy"])

In [16]:
with tf.device ('/gpu:0'):
    model.fit(x, y, batch_size=16 , epochs=60 )

Epoch 1/60
165/165 [==============================] - 48s 280ms/step - loss: 1.0307 - accuracy: 0.5870
Epoch 2/60
165/165 [==============================] - 46s 278ms/step - loss: 0.5639 - accuracy: 0.6955
Epoch 3/60
165/165 [==============================] - 46s 278ms/step - loss: 0.4872 - accuracy: 0.7596
Epoch 4/60
165/165 [==============================] - 46s 279ms/step - loss: 0.4513 - accuracy: 0.7740
Epoch 5/60
165/165 [==============================] - 46s 277ms/step - loss: 0.4645 - accuracy: 0.7600
Epoch 6/60
165/165 [==============================] - 47s 282ms/step - loss: 0.4353 - accuracy: 0.7922
Epoch 7/60
165/165 [==============================] - 45s 271ms/step - loss: 0.3824 - accuracy: 0.8100
Epoch 8/60
165/165 [==============================] - 43s 261ms/step - loss: 0.3839 - accuracy: 0.8051
Epoch 9/60
165/165 [==============================] - 43s 260ms/step - loss: 0.3669 - accuracy: 0.8225
Epoch 10/60
165/165 [==============================] - 42s 257ms/step - l

In [17]:
loss, accuracy = model.evaluate(x_test, y_test)

print("Test Accuracy:", accuracy)

21/21 [==============================] - 2s 72ms/step - loss: 1.0556 - accuracy: 0.8409
Test Accuracy: 0.8409090638160706
